# HyperSHAP: Example for Tunability Explanations

In this example, we consider the scenario where we want to explain the tunability of hyperparameters with respect to a baseline configuration and analyze to what extent tuning (sets of) hyperparameters benefits performance.

## Mockup setup of environment
To this end, we first setup some basic environment, which is assumed to be already existing when working with HyperSHAP. We will setup a configuration space, a baseline configuration and a black box function for which we would like to analyze the tunability of the hyperparameters.

In [ ]:
from __future__ import annotations

import math

from ConfigSpace import Configuration, ConfigurationSpace

# Configuration space with a float, integer, and categorical hyperparameter
cs = ConfigurationSpace(
    name="myspace",
    space={
        "a": (0.1, 1.5),  # UniformFloat
        "b": (2, 10),  # UniformInt
        "c": ["X", "Y"],  # Categorical
    },
)


# Some evaluation function that assesses the performance of some configuration
def eval_fun(config: Configuration) -> float:
    if config["c"] == "X":
        return math.sin(config["a"]) + config["b"]
    if config["c"] == "Y":
        return math.cos(config["a"] * config["b"]) + 1.5
    return 0


cs.seed(42)  # set some random seed for reproducibility

# sample some configuration as a baseline
baseline_config = cs.sample_configuration()

print("Baseline", baseline_config, "\nValue", eval_fun(baseline_config))

## HyperSHAP Instantiation with Configuration Space and Surrogate Model

Now it is time to instantiate HyperSHAP. For this, we need to define an explanation task that consists of a configuration space and a surrogate model for predicting the performance of a configuration. `ExplanationTask` comes with a couple of static convenience functions that facilitate the transition from the above configuration space and eval function to an explanation task. For simply building a surrogate model from the specified evaluation function, we can simply call the `from_function()` method and give the resulting explanation task to HyperSHAP.

In [ ]:
from hypershap.hypershap import HyperSHAP
from hypershap.task import ExplanationTask

explanation_task = ExplanationTask.from_function(config_space=cs, function=eval_fun)
hypershap = HyperSHAP(explanation_task=explanation_task)

# HyperSHAP Tunability

In [ ]:
iv = hypershap.tunability(baseline_config=baseline_config)
print(iv)

## Plot Shapley Interactions Graph

Now that we have determined the interaction values of the ablation game, let us look into some visualizations of the scenario above.

In [ ]:
hypershap.plot_si_graph()